In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==========================================================
# 1) Helper: Extract regional monthly mean FG_CO2
# ==========================================================
def extract_fgco2_monthlymean(ds_fg, lat_pt=31, lon_range=(140, 165)):
    """
    Extract monthly mean FG_CO2 (air–sea CO₂ flux)
    Positive upward (ocean → atmosphere).
    """
    da = ds_fg.sel(lat=lat_pt, method="nearest").sel(lon=slice(*lon_range))
    valid_mask = da.notnull().all(dim="lon")
    mean_val = da.mean(dim="lon", skipna=False).where(valid_mask)
    return mean_val.groupby("time.month").mean(dim="time")


# ==========================================================
# 2) Extract data for each simulation
# ==========================================================
fg_LE  = extract_fgco2_monthlymean(-cfg_var_FG_CO2.LE_ds_rgd)
fg_ADA = extract_fgco2_monthlymean(-cfg_var_FG_CO2.ADA_ds_rgd)
fg_WDA = extract_fgco2_monthlymean(-cfg_var_FG_CO2.WDA_ds_rgd)
fg_ODA = extract_fgco2_monthlymean(-cfg_var_FG_CO2.ODA_ds_rgd)

# ==========================================================
# 3) Compute Feb / Aug / Annual mean values
# ==========================================================
FG_Feb = {k: float(v.sel(month=2)) for k, v in {"LE": fg_LE, "ADA": fg_ADA, "WDA": fg_WDA, "ODA": fg_ODA}.items()}
FG_Aug = {k: float(v.sel(month=8)) for k, v in {"LE": fg_LE, "ADA": fg_ADA, "WDA": fg_WDA, "ODA": fg_ODA}.items()}
FG_Ann = {k: float(v.mean(dim="month")) for k, v in {"LE": fg_LE, "ADA": fg_ADA, "WDA": fg_WDA, "ODA": fg_ODA}.items()}

# ==========================================================
# 4) Compute differences (Exp − LE)
# ==========================================================
def compute_diff(dic):
    return {
        "ADA−LE": dic["ADA"] - dic["LE"],
        "WDA−LE": dic["WDA"] - dic["LE"],
        "ODA−LE": dic["ODA"] - dic["LE"],
    }

diff_Feb = compute_diff(FG_Feb)
diff_Aug = compute_diff(FG_Aug)
diff_Ann = compute_diff(FG_Ann)

# ==========================================================
# 5) Plot bar chart (3×2 panels: Feb, Aug, Annual)
# ==========================================================
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 12))

exps = ["LE", "ADA", "WDA", "ODA"]
diffs = ["ADA−LE", "WDA−LE", "ODA−LE"]
colors = ["k", "C3", "C0", "C2"]

# ========== February ==========
axes[0,0].bar(exps, [FG_Feb[k] for k in exps], color=colors)
axes[0,0].set_title("Feb FG_CO₂ (regional mean)")
axes[0,0].set_ylabel("Flux (kgC m⁻² s⁻¹)")

axes[0,1].bar(diffs, [diff_Feb[k] for k in diffs], color=["C3", "C0", "C2"])
axes[0,1].set_title("Feb Sea-to-air flux (Exp − LE)")

# ========== August ==========
axes[1,0].bar(exps, [FG_Aug[k] for k in exps], color=colors)
axes[1,0].set_title("Aug FG_CO₂ (regional mean)")
axes[1,0].set_ylabel("Flux (kgC m⁻² s⁻¹)")

axes[1,1].bar(diffs, [diff_Aug[k] for k in diffs], color=["C3", "C0", "C2"])
axes[1,1].set_title("Aug Sea-to-air flux (Exp − LE)")

# ========== Annual ==========
axes[2,0].bar(exps, [FG_Ann[k] for k in exps], color=colors)
axes[2,0].set_title("Annual FG_CO₂ (regional mean)")
axes[2,0].set_ylabel("Flux (kgC m⁻² s⁻¹)")

axes[2,1].bar(diffs, [diff_Ann[k] for k in diffs], color=["C3", "C0", "C2"])
axes[2,1].set_title("Annual Sea-to-air flux (Exp − LE)")

# ========== Formatting ==========
for i in range(3):
    for j in range(2):
        axes[i,j].grid(axis="y", ls="--", alpha=0.5)
        axes[i,j].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()
